In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score
from Constants import *
from Preprocessing import *
from Models import *
pd.set_option('display.max_rows', 200)



../data/models/


In [59]:
data = DTDataset(use_smote=True,smote_ids = Const.stratified_train_ids)
data.processed_df.shape, len(data.processed_df.index.unique())

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


here


((2186, 111), 2186)

In [4]:
model_dir = Const.data_dir + '/models/'

tuned_transition_models = [
    'final_transition1_model_state1_input63_dims500,500_dropout0.25,0.5.pt',
    'final_transition2_model_state2_input85_dims100_dropout0.25,0.pt',
    'final_outcome_model_state1_input83_dims1000_dropout0,0.pt'
]
tuned_transition_models = [model_dir + f for f in tuned_transition_models]
Const.tuned_transition_models = tuned_transition_models
Const.tuned_decision_model = model_dir +  'final_decision_model_statedecisions_input132_dims100,100_dropout0.1,0.7.pt'

In [42]:
def get_dt_ids():
    df = load_digital_twin()
    return df.id.values

def get_tt_split(ids=None,use_default_split=True,use_bagging_split=False,resample_training=False):
        if ids is None:
            ids = get_dt_ids()
        #pre-made, stratified by decision and outcome 72:28
        if use_default_split:
            train_ids = Const.stratified_train_ids[:]
            test_ids = Const.stratified_test_ids[:]
        elif use_bagging_split:
            train_ids = np.random.choice(ids,len(ids),replace=True)
            test_ids = [i for i in ids if i not in train_ids]
        else:
            test_ids = ids[0: int(len(ids)*(1-split))]
            train_ids = [i for i in ids if i not in test_ids]

        if resample_training:
            train_ids = np.random.choice(train_ids,len(train_ids),replace=True)
            test_ids = [i for i in ids if i not in train_ids]
        return train_ids,test_ids
    
def df_to_torch(df,ttype  = torch.FloatTensor):
    values = df.values.astype(float)
    values = torch.from_numpy(values)
    return values.type(ttype)

In [43]:
def nllloss(ytrue,ypred):
    #nll loss with argmax added in
    loss = torch.nn.NLLLoss()
    return loss(ypred,ytrue.argmax(axis=1))

def state_loss(ytrue,ypred,weights=[1,1,1,1]):
    pd_loss = nllloss(ytrue[0],ypred[0])*weights[0]
    nd_loss = nllloss(ytrue[1],ypred[1])*weights[1]
    mod_loss = nllloss(ytrue[2],ypred[2])*weights[2]
    loss = pd_loss + nd_loss + mod_loss
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
#     nloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    for i in range(ndlt):
        dlt_loss = bce(dlt_pred[:,i].view(-1),dlt_true[:,i].view(-1))
        loss += dlt_loss*weights[3]/ndlt
    return loss

def outcome_loss(ytrue,ypred,weights=[1,1,1]):
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[i])*weights[i]
        loss += iloss
    return loss

def mc_metrics(yt,yp,numpy=False,is_dlt=False):
    if not numpy:
        yt = yt .cpu().detach().numpy()
        yp = yp.cpu().detach().numpy()
    #dlt prediction (binary)
    if is_dlt:
        acc = accuracy_score(yt,yp>.5)
        if yt.sum() > 1:
            auc = roc_auc_score(yt,yp)
        else:
            auc=-1
        error = np.mean((yt-yp)**2)
        return {'accuracy': acc, 'mse': error, 'auc': auc}
    #this is a catch for when I se the dlt prediction format (encoded integer ordinal, predict as a categorical and take the argmax)
    elif yt.ndim > 1:
        try:
            bacc = balanced_accuracy_score(yt.argmax(axis=1),yp.argmax(axis=1))
        except:
            bacc = -1
        try:
            roc_micro = roc_auc_score(yt,yp,average='micro')
        except:
            roc_micro=-1
        try:
            roc_macro = roc_auc_score(yt,yp,average='macro')
        except:
            roc_macro = -1
        return {'accuracy': bacc, 'roc_micro': roc_micro,'roc_macro': roc_macro}
    #outcomes (binary)
    else:
        if yp.ndim > 1:
            yp = yp.argmax(axis=1)
        try:
            bacc = accuracy_score(yt,yp)
        except:
            bacc = -1
        try:
            roc = roc_auc_score(yt,yp)
        except:
            roc = -1
        error = np.mean((yt-yp)**2)
        return {'accuracy': bacc, 'mse': error, 'auc': roc}

def state_metrics(ytrue,ypred,numpy=False):
    pd_metrics = mc_metrics(ytrue[0],ypred[0],numpy=numpy)
    nd_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    mod_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    
    dlt_metrics = []
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
    nloss = torch.nn.NLLLoss()
    for i in range(ndlt):
        dm = mc_metrics(dlt_true[:,i],dlt_pred[:,i].view(-1),is_dlt=True)
        dlt_metrics.append(dm)
    dlt_acc =[d['accuracy'] for d in dlt_metrics]
    dlt_error = [d['mse'] for d in dlt_metrics]
    dlt_auc = [d['auc'] for d in dlt_metrics]
    return {'pd': pd_metrics,'nd': nd_metrics,'mod': mod_metrics,'dlts': {'accuracy': dlt_acc,'accuracy_mean': np.mean(dlt_acc),'auc': dlt_auc,'auc_mean': np.mean(dlt_auc)}}
    
def outcome_metrics(ytrue,ypred,numpy=False):
    res = {}
    for i, outcome in enumerate(Const.outcomes):
        metrics = mc_metrics(ytrue[i],ypred[:,i])
        res[outcome] = metrics
    return res


In [44]:
from sklearn.ensemble import RandomForestClassifier

def train_state_rf(model_args={}):
    ids = get_dt_ids()
    
    dataset = DTDataset()

    train_ids = ids[0:int(len(ids)*.7)]
    test_ids = ids[int(len(ids)*.7):]
    
    #most things are multiclass, dlts are several ordinal and outcomes are multiple binary
    xtrain1 = dataset.get_state('baseline',ids=train_ids)
    xtest1 = dataset.get_state('baseline',ids=test_ids)
    
    xtrain2 = dataset.get_input_state(step=2,ids=train_ids)
    xtest2 = dataset.get_input_state(step=2,ids=test_ids)
    
    xtrain3 = dataset.get_input_state(step=3,ids=train_ids)
    xtest3 = dataset.get_input_state(step=3,ids=test_ids)
    
    [pd1_train,nd1_train, mod_train,dlts1_train] = dataset.get_intermediate_outcomes(ids=train_ids)
    [pd2_train,nd2_train, cc_train,dlts2_train] = dataset.get_intermediate_outcomes(step=2,ids=train_ids)
    [pd1_test,nd1_test, mod_test,dlts1_test] = dataset.get_intermediate_outcomes(ids=test_ids)
    [pd2_test,nd2_test, cc_test,dlts2_test] = dataset.get_intermediate_outcomes(step=2,ids=test_ids)
    outcomes_train = dataset.get_state('outcomes',ids=train_ids)
    outcomes_test = dataset.get_state('outcomes',ids=test_ids)
    

    def train_multiclass_rf(xtrain,xtest,ytrain,ytest):
        model = RandomForestClassifier(class_weight='balanced',**model_args).fit(xtrain,ytrain)
        ypred = model.predict(xtest)
        metrics = mc_metrics(ytest.values,ypred,numpy=True)
        return model, metrics
    
    all_metrics = {}
    pd1_model, all_metrics['pd1'] = train_multiclass_rf(xtrain1,xtest1,pd1_train,pd1_test)
    nd1_model, all_metrics['nd1']  = train_multiclass_rf(xtrain1,xtest1,nd1_train,nd1_test)
    mod_model, all_metrics['mod']  = train_multiclass_rf(xtrain1,xtest1,mod_train,mod_test)
    
    return all_metrics

train_state_rf({'max_depth': 5,'n_estimators': 100})

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  


Index(['hpv', 'age', 'packs_per_year', 'smoking_status', 'gender',
       'Aspiration rate Pre-therapy', 'total_dose', 'dose_fraction', '1A1B',
       '1A6',
       ...
       'SD Primary 2', 'CR Nodal 2', 'PR Nodal 2', 'SD Nodal 2',
       'Decision 1 (Induction Chemo) Y/N', 'Decision 2 (CC / RT alone)',
       'Decision 3 Neck Dissection (Y/N)', 'Overall Survival (4 Years)', 'FT',
       'Aspiration rate Post-therapy'],
      dtype='object', length=111)


{'pd1': {'accuracy': 0.36326291079812206,
  'roc_micro': 0.5126910126910126,
  'roc_macro': 0.5150862068965517},
 'nd1': {'accuracy': 0.38838037121789987,
  'roc_micro': 0.5717430717430717,
  'roc_macro': 0.5260869565217391},
 'mod': {'accuracy': 0.16666666666666666,
  'roc_micro': 0.515527950310559,
  'roc_macro': -1}}

In [45]:
 DTDataset().get_state('baseline').T

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


Index(['hpv', 'age', 'packs_per_year', 'smoking_status', 'gender',
       'Aspiration rate Pre-therapy', 'total_dose', 'dose_fraction', '1A1B',
       '1A6',
       ...
       'SD Primary 2', 'CR Nodal 2', 'PR Nodal 2', 'SD Nodal 2',
       'Decision 1 (Induction Chemo) Y/N', 'Decision 2 (CC / RT alone)',
       'Decision 3 Neck Dissection (Y/N)', 'Overall Survival (4 Years)', 'FT',
       'Aspiration rate Post-therapy'],
      dtype='object', length=111)


id,3,5,6,7,8,9,10,11,13,14,...,10196,10197,10198,10199,10200,10201,10202,10203,10204,10205
1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1A1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1A6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1B2A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2A,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
2A2B,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
2A3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2B,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [155]:
def train_state(model=None,
                model_args={},
                state=1,
                split=.7,
                lr=.0001,
                epochs=1000,
                patience=10,
                use_attention=True,
                weights=[1,1,1,10],
                save_path='../data/models/',
                use_default_split=True,
                use_bagging_split=False,
                resample_training=False,#use bootstraping on training data after splitting
                n_validation_trainsteps=2,
                verbose=True,
                use_smote=True,
                file_suffix=''):
    
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    
    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()

    xtrain = dataset.get_input_state(step=state,ids=train_ids)
    xtest = dataset.get_input_state(step=state,ids=test_ids)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids)
    

    if not use_attention:
        model_args = {k:v for k,v in model_args.items() if 'attention' not in k and 'embed_size' not in k}
    if state < 3:
        if model is None:
            if use_attention:
                model = OutcomeAttentionSimulator(xtrain.shape[1],state=state,**model_args)
            else:
                model = OutcomeSimulator(xtrain.shape[1],state=state,**model_args)
        lfunc = state_loss
    else:
        if model is None:
            if use_attention:
                model = EndpointAttentionSimulator(xtrain.shape[1],**model_args)
            else:
                model = EndpointSimulator(xtrain.shape[1],**model_args)
        weights = weights[:3]
        lfunc = outcome_loss
        
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    save_file = save_path + 'model_' + model.identifier + '_split' + str(split) + '_resample' + str(resample_training) +  '_hash' + hashcode + file_suffix + '.tar'
    xtrain = df_to_torch(xtrain)
    xtest = df_to_torch(xtest)
    ytrain = [df_to_torch(t) for t in ytrain]
    ytest= [df_to_torch(t) for t in ytest]
    
    model.fit_normalizer(xtrain)
#     normalize = lambda x: (x - xtrain.mean(axis=0)+.01)/(xtrain.std(axis=0)+.01)
#     unnormalize = lambda x: (x * (xtrain.std(axis=0) +.01)) + xtrain.mean(axis=0) - .01
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_val_loss = 1000000000000000000000000000
    best_loss_metrics = {}
    last_epoch = False
    for epoch in range(epochs):
        
        model.train(True)
        optimizer.zero_grad()
        
        xtrain_sample = xtrain#[torch.randint(len(xtrain),(len(xtrain),) )]
        ypred = model(xtrain_sample)
        loss = lfunc(ytrain,ypred,weights=weights)

        loss.backward()
        optimizer.step()
        if verbose:
            print('epoch',epoch,'train loss',loss.item())
        
        model.eval()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        if state < 3:
            val_metrics = state_metrics(ytest,yval)
        else:
            val_metrics = outcome_metrics(ytest,yval)
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_loss_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if verbose:
            print('val loss',val_loss.item())
            print('______________')
        if steps_since_improvement > patience:
            break
    print('best loss',best_val_loss,best_loss_metrics)
    model.load_state_dict(torch.load(save_file))
    
    #train one step on validation data
    for i in range(n_validation_trainsteps):
        model.train()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        val_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),save_file)
    
    model.eval()
    return model,  best_val_loss, best_loss_metrics

model1,_,_ = train_state(model_args=Const.optimized_model_parameters['transition1'],use_attention=False)
model1

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
epoch 0 train loss 14.432960510253906
val loss 11.280978202819824
______________
epoch 1 train loss 14.363431930541992
val loss 11.16526985168457
______________
epoch 2 train loss 14.05457878112793
val loss 11.051637649536133
______________
epoch 3 train loss 14.123756408691406
val loss 10.939537048339844
______________
epoch 4 train loss 13.916208267211914
val loss 10.829231262207031
______________
epoch 5 train loss 13.62899112701416
val loss 10.720664024353027
______________
epoch 6 train loss 13.720484733581543
val loss 10.61370849609375
______________
epoch 7 train loss 13.422891616821289
val loss 10.508469581604004
______________
epoch 8 train loss 13.6050386428833
val loss 10.404630661010742
______________
epoch 9 train loss 13.218210220336914
val loss 10.30260944366455
______________
epoch 10 train loss 13.140597343444824
val loss 10.20211410522461
______________
epoch 11 train loss 13.193403244018555
val loss 10.103120803833008
______________
epoch 12 train loss 12.954442

epoch 103 train loss 6.650635719299316
val loss 5.5782790184021
______________
epoch 104 train loss 6.592278003692627
val loss 5.556732654571533
______________
epoch 105 train loss 6.642437934875488
val loss 5.535426139831543
______________
epoch 106 train loss 6.5776143074035645
val loss 5.51439905166626
______________
epoch 107 train loss 6.485738754272461
val loss 5.493669033050537
______________
epoch 108 train loss 6.4265313148498535
val loss 5.47321891784668
______________
epoch 109 train loss 6.37692403793335
val loss 5.452978134155273
______________
epoch 110 train loss 6.443424224853516
val loss 5.43297004699707
______________
epoch 111 train loss 6.349940776824951
val loss 5.413232326507568
______________
epoch 112 train loss 6.3595757484436035
val loss 5.393763065338135
______________
epoch 113 train loss 6.187060356140137
val loss 5.374607086181641
______________
epoch 114 train loss 6.111922740936279
val loss 5.355743885040283
______________
epoch 115 train loss 6.22322368

KeyboardInterrupt: 

In [ ]:
model2,_,_ = train_state(model_args=Const.optimized_model_parameters['transition2'],state=2,use_attention=False)
model2

In [ ]:
model3,_,_ = train_state(model_args=Const.optimized_model_parameters['outcomes'],state=3,use_attention=False)
model3

In [156]:
def gridsearch_attention_transition_models(state=1,attentions=[True,False]):
    model_arglist = [
        {
            'hidden_layers': [100],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [500],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [10],
        },
        {
            'hidden_layers': [100],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [500],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [10,10],
        },
        {
            'hidden_layers': [500,500],
            'attention_heads': [10,10],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [5,5],
        },
        {
            'hidden_layers': [500,500],
            'attention_heads': [5,5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for use_attention in attentions:
            embed_sizes = [200,400] if use_attention else [0]
            for embed_size in embed_sizes:
                args['embed_size'] = embed_size
                for dropout in [.9,]:
                    args['dropout'] = dropout
                    for input_dropout in [.5]:
                        args['input_dropout'] = input_dropout
                    
                        model,m_loss,m_metrics = train_state(model_args=args,state=state,use_attention=use_attention,verbose=False)
                        print('done',k,m_loss)
                        k+=1
                        if m_loss < best_loss:
                            best_loss = m_loss
                            best_metrics  = m_metrics
                            best_model = model
                            best_args = args
                            print('_++++++++++New Best++++____')
                            print(best_loss)
                            print(best_metrics)
                            print(best_args)
                            print(use_attention)
                            print('___________')
                            print('++++++++')
                            print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model
model1 = gridsearch_attention_transition_models(1)
model1

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.764965534210205 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6839781628293378, 'roc_macro': 0.6330226813097114}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.9510489510489512, 0.9313131313131313, 0.9527363184079602, -1, 0.9092198581560285, -1, 0.9333333333333333, -1], 'auc_mean': 0.20970644903242558}}
done 0 2.764965534210205
_++++++++++New Best++++____
2.764965534210205
{'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6839781628293378, 'roc_macro': 0.6330226813097114}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'mod': {'accuracy': 0.333333333333

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 4.025295257568359 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6755281272252552, 'roc_macro': 0.6340827355168788}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.5679000245037981, 'roc_macro': 0.559701470027557}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.5679000245037981, 'roc_macro': 0.559701470027557}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.766899766899767, 0.8424242424242425, 0.7170398009950248, -1, 0.75177304964539, -1, 0.8880952380952382, -1], 'auc_mean': 0.12077901225745781}}
done 1 4.025295257568359


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.7161152362823486 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6835509138381202, 'roc_macro': 0.6361547246381728}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6163195295270767, 'roc_macro': 0.6741712393886307}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6163195295270767, 'roc_macro': 0.6741712393886307}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.9137529137529138, 0.9683501683501683, 0.9359452736318408, -1, 0.9304964539007092, -1, 0.9011904761904762, -1], 'auc_mean': 0.20621691072826354}}
done 28 2.7161152362823486


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.3858258724212646 {'pd': {'accuracy': 0.4969654528478058, 'roc_micro': 0.6851174934725849, 'roc_macro': 0.6237181607808212}, 'nd': {'accuracy': 0.5198805594845198, 'roc_micro': 0.7011026709139917, 'roc_macro': 0.6828778105952019}, 'mod': {'accuracy': 0.5198805594845198, 'roc_micro': 0.7011026709139917, 'roc_macro': 0.6828778105952019}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.8951048951048951, 0.9555555555555556, 0.9253731343283582, -1, 0.9432624113475176, -1, 0.8952380952380953, -1], 'auc_mean': 0.20181676144680272}}
done 29 2.3858258724212646
_________
+++++++++++
best stuff 2.251129150390625
{'pd': {'accuracy': 0.375, 'roc_micro': 0.6783764538333729, 'roc_macro': 0.6248571736860679}, 'nd': {'accuracy': 0.4801980198019802, 'roc_micro': 0.6757167360940945, 'roc_macro': 0.6780191426930556}, 'mod': {'accuracy': 0.4801980198019802,

OutcomeAttentionSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=100, out_features=1000, bias=True)
  )
  (batchnorm): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (resize_layer): Linear(in_features=63, out_features=100, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  )
  (final_layer): Linear(in_features=1000, out_features=3, bias=True)
  (activation): ReLU()
  (disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (nodal_disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (dlt_layers): ModuleList(
    (0): Linear(in_features=1000, out_features=1

In [157]:
model2 = gridsearch_attention_transition_models(2)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains clas

best loss 3.808809757232666 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9072182055232902, 'roc_macro': -1}, 'nd': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'mod': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.2564102564102564, 0.5926616915422885, 0.60431654676259, -1, 0.5035460992907801, -1, 0.5943262411347519, -1], 'auc_mean': -0.05609239560741665}}
done 0 3.808809757232666
_++++++++++New Best++++____
3.808809757232666
{'pd': {'accuracy': 0.5, 'roc_micro': 0.9072182055232902, 'roc_macro': -1}, 'nd': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'mod': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.474197131209326

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

best loss 3.8991506099700928 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9260874718501837, 'roc_macro': -1}, 'nd': {'accuracy': 0.32383966244725737, 'roc_micro': 0.7193041553379371, 'roc_macro': 0.484346802563266}, 'mod': {'accuracy': 0.32383966244725737, 'roc_micro': 0.7193041553379371, 'roc_macro': 0.484346802563266}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.48717948717948717, 0.5690298507462687, 0.4696813977389517, -1, 0.5397163120567376, -1, 0.3588652482269504, -1], 'auc_mean': -0.07194096300645057}}
done 28 3.8991506099700928


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains clas

best loss 3.4415688514709473 {'pd': {'accuracy': 0.4959349593495935, 'roc_micro': 0.9374659239066019, 'roc_macro': -1}, 'nd': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'mod': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.3776223776223776, 0.595771144278607, 0.6402877697841727, -1, 0.6624113475177306, -1, 0.3801418439716312, -1], 'auc_mean': -0.04297068960318512}}
done 29 3.4415688514709473
_++++++++++New Best++++____
3.4415688514709473
{'pd': {'accuracy': 0.4959349593495935, 'roc_micro': 0.9374659239066019, 'roc_macro': -1}, 'nd': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'mod': {'accuracy': 0.34236550632911394, 'roc_micro': 0.75236399

In [158]:
model3 = gridsearch_attention_transition_models(3)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.9533352851867676 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.22294936, 'auc': 0.61875}, 'FT': {'accuracy': -1, 'mse': 0.22988118, 'auc': 0.6208881578947368}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.23439899, 'auc': 0.7096153846153845}}
done 0 1.9533352851867676
_++++++++++New Best++++____
1.9533352851867676
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.22294936, 'auc': 0.61875}, 'FT': {'accuracy': -1, 'mse': 0.22988118, 'auc': 0.6208881578947368}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.23439899, 'auc': 0.7096153846153845}}
{'hidden_layers': [100], 'attention_heads': [1], 'embed_size': 200, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.9169039726257324 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.23428532, 'auc': 0.5783653846153846}, 'FT': {'accuracy': -1, 'mse': 0.21831562, 'auc': 0.6165021929824561}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.21789795, 'auc': 0.7125}}
done 1 1.9169039726257324
_++++++++++New Best++++____
1.9169039726257324
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.23428532, 'auc': 0.5783653846153846}, 'FT': {'accuracy': -1, 'mse': 0.21831562, 'auc': 0.6165021929824561}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.21789795, 'auc': 0.7125}}
{'hidden_layers': [100], 'attention_heads': [1], 'embed_size': 400, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

here
best loss 1.513304352760315 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.15192407, 'auc': 0.6110576923076922}, 'FT': {'accuracy': -1, 'mse': 0.19458365, 'auc': 0.5923793859649122}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14482027, 'auc': 0.7080128205128204}}
done 27 1.513304352760315


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.455763339996338 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
done 28 1.455763339996338
_++++++++++New Best++++____
1.455763339996338
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
{'hidden_layers': [500, 500], 'attention_heads': [5, 5], 'embed_size': 400, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.4676107168197632 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12858523, 'auc': 0.6384615384615384}, 'FT': {'accuracy': -1, 'mse': 0.19938947, 'auc': 0.6077302631578947}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.13879225, 'auc': 0.7153846153846154}}
done 29 1.4676107168197632
_________
+++++++++++
best stuff 1.455763339996338
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
{'hidden_layers': [500, 500], 'attention_heads': [5, 5], 'embed_size': 0, 'dropout': 0.9, 'input_dropout': 0.5}


In [ ]:
def load_trained_models():
    files = Const.tuned_transition_models
    decision_file = Const.tuned_decision_model
    [model1,model2,model3] = [torch.load(file) for file in files]
    decision_model = torch.load(decision_file)
    return decision_model, model1,model2,model3
_, model1, model2, model3 =load_trained_models()

In [159]:

def shuffle_col(v,col=None):
    if col is None:
        col = np.random.choice([i for i in range(v.shape[1])])
    idx = torch.randperm(v.shape[0])
    vv = torch.clone(v)
    vv[:,col] = vv[idx,col]
    return vv
    
def train_decision_model(
    tmodel1,
    tmodel2,
    tmodel3,
    use_default_split=True,
    use_bagging_split=False,
    lr=.0001,
    epochs=10000,
    patience=100,
    weights=[3,1,1], #realtive weight of survival, feeding tube, and aspiration
    imitation_weight=1,
    shufflecol_chance = 0.1,
    reward_weight=10,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    use_attention=False,
    verbose=True,
    threshold_decisions=True,
    use_smote=True,
    **model_kwargs,
):
    
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)

    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()

    data = dataset.processed_df.copy()
    
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids])
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model = DecisionModel(baseline.shape[1],**model_kwargs)

    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)

    def outcome_loss(ypred):
        #convert survival to death
        loss = torch.mul(torch.mean(-1*(ypred[:,0] - 1)),weights[0])
        for i,weight in enumerate(weights[1:]):
            newloss = torch.mean(ypred[:,i])*weight
            loss = torch.add(loss,torch.mul(newloss,weight))
        return loss
    
    mse = torch.nn.MSELoss()
    nllloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids)))
    threshold = lambda x: torch.gt(x,.5).type(torch.FloatTensor)

    
    #save the inputs from the whole dataset for future reference
    full_data = []
    for mstep in [0,1,2]:
        full_data_step = [baseline, get_dlt(min(mstep,1)),
                     get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
        full_data_step = torch.cat([formatdf(fd,train_ids+test_ids) for fd in full_data_step],axis=1)
        full_data.append(full_data_step)
    full_data = torch.stack(full_data)
    model.save_memory(full_data)

    def step(train=True):
        if train:
            model.train(True)
            optimizer.zero_grad()
            ids = train_ids
        else:
            ids = test_ids
            model.eval()
            
            
        ytrain = df_to_torch(outcomedf.loc[ids])
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = [formatdf(xx,ids) for xx in xxtrained]
        o1 = model(torch.cat(xxtrain,axis=1),position=0)
            
        decision1_imitation = o1[:,3]
        decision1 = o1[:,0]
        if threshold_decisions:
            decision1 = threshold(decision1)
#         imitation_loss1 = bce(threshold(decision1_imitation),ytrain[:,0])
        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1)[:,4]
        
#         imitation_loss2 =  bce(threshold(decision2_imitation),ytrain[:,1])
        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
#         if model.keys is None:
#             keyvals = torch.cat([formatdf(xxx,train_ids+test_ids) for xxx in x2_imitation],axis=1)
#             model.set_keys(keyvals)
            
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        
        decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2)[:,5]
        
#         imitation_loss3 = bce(threshold(decision3_imitation),ytrain[:,2])
        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        
        #reward decisions
        xx1 = makeinput(1,ids)
        xx2 = makeinput(2,ids)
        xx3 = makeinput(3,ids)

        baseline_train_base = formatdf(baseline,ids)
            
        baseline_train = torch.clone(baseline_train_base)
        if train and shufflecol_chance > 0.0001:
            for col in range(baseline_train_base.shape[1]): 
                if np.random.random() < shufflecol_chance:
                    baseline_train = shuffle_col(baseline_train,col)
                    
        
        xi1 = torch.cat([xx1,decision1.view(-1,1)],axis=1)
        [ypd1, ynd1, ymod, ydlt1] = tmodel1(xi1)
        #this outputs log likelihoods (except for dlts) -> convert to probability
        ypd1 = torch.exp(ypd1)
        ynd1 = torch.exp(ynd1)
        ymod = torch.exp(ymod)
        x1 = [baseline_train,ydlt1,formatdf(get_dlt(0),ids),ypd1,ynd1,formatdf(get_cc(1),ids),ymod]
        
        decision2 = model(torch.cat(x1,axis=1),position=1)[:,1] 
        if threshold_decisions:
            decision2 = threshold(decision2)
            
        xi2 = torch.cat([xx2,decision1.view(-1,1),decision2.view(-1,1)],axis=1)
        [ypd2,ynd2,ycc,ydlt2] = tmodel2(xi2)
        ypd2 = torch.exp(ypd2)
        ynd2 = torch.exp(ynd2)
        ycc = torch.exp(ycc)
        x2 = [baseline_train,ydlt1,ydlt2,ypd2,ynd2,ycc,ymod]
            
        decision3 = model(torch.cat(x2,axis=1),position=2)[:,2]
        if threshold_decisions:
            decision3 = threshold(decision3)
            
        
        xi3 = torch.cat([xx3,decision1.view(-1,1),decision2.view(-1,1),decision3.view(-1,1)],axis=1)
        outcomes = tmodel3(xi3)

        reward_loss = outcome_loss(outcomes)
        loss = torch.add(imitation_loss1,imitation_loss2)
        loss = torch.add(loss,imitation_loss3)
        loss = torch.mul(loss,imitation_weight/3)
        loss = torch.add(loss,torch.mul(reward_loss,reward_weight))
        losses = [imitation_loss1+imitation_loss2+imitation_loss3,reward_loss]
        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            for i,decision in enumerate([decision1_imitation,decision2_imitation,decision3_imitation]):
                dec = decision.cpu().detach().numpy()
                dec0 = (dec > .5).astype(int)
                out = ytrain[:,i].cpu().detach().numpy()
                acc = accuracy_score(out,dec > .5)
                auc = roc_auc_score(out,dec)
                scores.append({'decision': i,'accuracy': acc,'auc': auc})
            return losses, scores
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics = step(False)
        vl = val_losses[0] + val_losses[1]
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('train imitation',losses[0].item(),'reward',losses[1].item())
            print('val imitation',val_losses[0].item(),'reward',val_losses[1].item())
            print('val loss',vl.item(),best_val_loss.item())
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss

from Models import *
args = {
    'hidden_layers': [600], 
    'attention_heads': [3], 
    'embed_size': 210, 
    'dropout': 0.9, 
    'input_dropout': 0.5, 
    'shufflecol_chance': 0.5,
}
decision_model, _, _ = train_decision_model(model1,model2,model3,lr=.0001,use_attention=True,**args)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
______epoch 0 _____
train imitation 2.2124528884887695 reward 2.1499383449554443
val imitation 2.0197110176086426 reward 1.594883918762207
val loss 3.6145949363708496 1000000000.0
[{'decision': 0, 'accuracy': 0.410958904109589, 'auc': 0.54375}, {'decision': 1, 'accuracy': 0.5684931506849316, 'auc': 0.5172697368421053}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.49615384615384617}]
______epoch 1 _____
train imitation 2.212952136993408 reward 2.147641181945801
val imitation 2.020482301712036 reward 1.5938901901245117
val loss 3.614372491836548 3.6145949363708496
[{'decision': 0, 'accuracy': 0.4383561643835616, 'auc': 0.5432692307692307}, {'decision': 1, 'accuracy': 0.5616438356164384, 'auc': 0.5194627192982456}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.5064102564102564}]
______epoch 2 _____
train imitation 2.191307306289673 reward 2.1485722064971924
val imitation 2.021440267562866 reward 1.5932416915893555
val loss 3.6146819591522217 3.614372491836548
[

______epoch 20 _____
train imitation 2.0754218101501465 reward 2.147006034851074
val imitation 2.0003154277801514 reward 1.59486985206604
val loss 3.5951852798461914 3.59871768951416
[{'decision': 0, 'accuracy': 0.7602739726027398, 'auc': 0.5408653846153846}, {'decision': 1, 'accuracy': 0.547945205479452, 'auc': 0.5550986842105263}, {'decision': 2, 'accuracy': 0.6712328767123288, 'auc': 0.6548076923076923}]
______epoch 21 _____
train imitation 2.0758228302001953 reward 2.1440460681915283
val imitation 1.9967174530029297 reward 1.5939974784851074
val loss 3.590714931488037 3.5951852798461914
[{'decision': 0, 'accuracy': 0.773972602739726, 'auc': 0.5403846153846155}, {'decision': 1, 'accuracy': 0.5547945205479452, 'auc': 0.5550986842105263}, {'decision': 2, 'accuracy': 0.6712328767123288, 'auc': 0.6592948717948718}]
______epoch 22 _____
train imitation 2.0940544605255127 reward 2.1487841606140137
val imitation 1.9932177066802979 reward 1.5942072868347168
val loss 3.5874249935150146 3.590

______epoch 40 _____
train imitation 2.028940200805664 reward 2.146420955657959
val imitation 1.907974123954773 reward 1.5914645195007324
val loss 3.499438762664795 3.504575729370117
[{'decision': 0, 'accuracy': 0.8424657534246576, 'auc': 0.5514423076923077}, {'decision': 1, 'accuracy': 0.5684931506849316, 'auc': 0.5819627192982456}, {'decision': 2, 'accuracy': 0.726027397260274, 'auc': 0.701602564102564}]
______epoch 41 _____
train imitation 2.0089199542999268 reward 2.1512644290924072
val imitation 1.9032403230667114 reward 1.5907930135726929
val loss 3.4940333366394043 3.499438762664795
[{'decision': 0, 'accuracy': 0.8424657534246576, 'auc': 0.5480769230769231}, {'decision': 1, 'accuracy': 0.5753424657534246, 'auc': 0.5836074561403508}, {'decision': 2, 'accuracy': 0.7328767123287672, 'auc': 0.7022435897435898}]
______epoch 42 _____
train imitation 2.010678768157959 reward 2.147810459136963
val imitation 1.898987889289856 reward 1.590831995010376
val loss 3.4898200035095215 3.4940333

______epoch 60 _____
train imitation 1.9541609287261963 reward 2.148322582244873
val imitation 1.813849687576294 reward 1.5918288230895996
val loss 3.4056785106658936 3.409555435180664
[{'decision': 0, 'accuracy': 0.8698630136986302, 'auc': 0.5389423076923077}, {'decision': 1, 'accuracy': 0.589041095890411, 'auc': 0.5959429824561404}, {'decision': 2, 'accuracy': 0.7534246575342466, 'auc': 0.7118589743589743}]
______epoch 61 _____
train imitation 1.9606118202209473 reward 2.1438863277435303
val imitation 1.8078949451446533 reward 1.5920405387878418
val loss 3.399935483932495 3.4056785106658936
[{'decision': 0, 'accuracy': 0.8698630136986302, 'auc': 0.5375}, {'decision': 1, 'accuracy': 0.589041095890411, 'auc': 0.5956688596491229}, {'decision': 2, 'accuracy': 0.7602739726027398, 'auc': 0.7112179487179487}]
______epoch 62 _____
train imitation 1.9488232135772705 reward 2.145174503326416
val imitation 1.8022876977920532 reward 1.593157172203064
val loss 3.395444869995117 3.399935483932495


...

______epoch 340 _____
train imitation 1.355359435081482 reward 2.150317430496216
val imitation 1.398505687713623 reward 1.6004762649536133
val loss 2.9989819526672363 2.9726643562316895
[{'decision': 0, 'accuracy': 0.8013698630136986, 'auc': 0.5139423076923076}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6891447368421053}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.7522435897435897}]
______epoch 341 _____
train imitation 1.386660099029541 reward 2.1506829261779785
val imitation 1.3983678817749023 reward 1.6011415719985962
val loss 2.999509334564209 2.9726643562316895
[{'decision': 0, 'accuracy': 0.8013698630136986, 'auc': 0.5139423076923076}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.689967105263158}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.7519230769230769}]
______epoch 342 _____
train imitation 1.3720617294311523 reward 2.1516735553741455
val imitation 1.3956995010375977 reward 1.6017608642578125
val loss 2.99746036529541 2.

______epoch 360 _____
train imitation 1.3709174394607544 reward 2.15177583694458
val imitation 1.387300729751587 reward 1.6033443212509155
val loss 2.990644931793213 2.9726643562316895
[{'decision': 0, 'accuracy': 0.8013698630136986, 'auc': 0.5192307692307693}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6921600877192982}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7503205128205128}]
______epoch 361 _____
train imitation 1.363262414932251 reward 2.152771234512329
val imitation 1.3862419128417969 reward 1.6029856204986572
val loss 2.989227533340454 2.9726643562316895
[{'decision': 0, 'accuracy': 0.8013698630136986, 'auc': 0.5211538461538461}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.6935307017543859}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.7496794871794872}]
______epoch 362 _____
train imitation 1.3526016473770142 reward 2.146758556365967
val imitation 1.3880075216293335 reward 1.60283362865448
val loss 2.9908411502838135 2.9726

______epoch 380 _____
train imitation 1.3541209697723389 reward 2.149705171585083
val imitation 1.4123632907867432 reward 1.6010165214538574
val loss 3.0133798122406006 2.9726643562316895
[{'decision': 0, 'accuracy': 0.7945205479452054, 'auc': 0.5153846153846153}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6896929824561404}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7516025641025641}]
______epoch 381 _____
train imitation 1.3196303844451904 reward 2.147414207458496
val imitation 1.4122800827026367 reward 1.6016751527786255
val loss 3.0139551162719727 2.9726643562316895
[{'decision': 0, 'accuracy': 0.7945205479452054, 'auc': 0.5153846153846154}, {'decision': 1, 'accuracy': 0.7602739726027398, 'auc': 0.689967105263158}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7516025641025641}]
______epoch 382 _____
train imitation 1.349907636642456 reward 2.146862745285034
val imitation 1.4134249687194824 reward 1.601707935333252
val loss 3.0151329040527344 2.

______epoch 400 _____
train imitation 1.303810715675354 reward 2.1487743854522705
val imitation 1.3934043645858765 reward 1.6018847227096558
val loss 2.9952890872955322 2.9726643562316895
[{'decision': 0, 'accuracy': 0.7945205479452054, 'auc': 0.5254807692307693}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6935307017543859}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.7522435897435897}]
______epoch 401 _____
train imitation 1.3005321025848389 reward 2.1490254402160645
val imitation 1.392671823501587 reward 1.6017216444015503
val loss 2.9943933486938477 2.9726643562316895
[{'decision': 0, 'accuracy': 0.7876712328767124, 'auc': 0.525}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.6951754385964912}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7532051282051282}]
______epoch 402 _____
train imitation 1.3074162006378174 reward 2.145726203918457
val imitation 1.3945155143737793 reward 1.6016699075698853
val loss 2.996185302734375 2.97266435623

______epoch 420 _____
train imitation 1.2829911708831787 reward 2.144697427749634
val imitation 1.4061720371246338 reward 1.6025927066802979
val loss 3.0087647438049316 2.9726643562316895
[{'decision': 0, 'accuracy': 0.821917808219178, 'auc': 0.5375000000000001}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6907894736842105}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.746474358974359}]
______epoch 421 _____
train imitation 1.3077857494354248 reward 2.1469521522521973
val imitation 1.4060169458389282 reward 1.6025927066802979
val loss 3.0086097717285156 2.9726643562316895
[{'decision': 0, 'accuracy': 0.8013698630136986, 'auc': 0.5375000000000001}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6910635964912281}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.7487179487179487}]
______epoch 422 _____
train imitation 1.2982474565505981 reward 2.1449079513549805
val imitation 1.402051329612732 reward 1.6018941402435303
val loss 3.003945350646972

In [66]:
def gridsearch_decision_model(m1,m2,m3):
    model_arglist = [

        {
            'hidden_layers': [100,100],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [50,50],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [2,2],
        },
        {
            'hidden_layers': [300],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [600],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [300,300],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [600,600],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for embed_size in [0,120,210]:
            #embed_size = 0 skips the firt layer that makes the sizes right
            if embed_size == 0 and args['attention_heads'][0] != 1:
                continue
            args['embed_size'] = embed_size
            for dropout in [.5,.9]:
                args['dropout'] = dropout
                for input_dropout in [.5]:
                    args['input_dropout'] = input_dropout
                    for shufflecol_chance in [.1,.5]:
                        args['shufflecol_chance'] = shufflecol_chance
                        model,m_metrics,m_loss = train_decision_model(m1,m2,m3,use_attention=True,verbose=False,**args)
                        print('done',k,m_loss)
                        print('curr best',best_loss)
                        k+=1
                        if m_loss < best_loss:
                            best_loss = m_loss
                            best_metrics  = m_metrics
                            best_model = model
                            best_args = args
                            print('_++++++++++New Best++++____')
                            print(best_loss)
                            print(best_metrics)
                            print(best_args)
                            print('___________')
                            print('++++++++')
                            print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model

from Models import *
decision_model = gridsearch_decision_model(model1,model2,model3)
decision_model

/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.7922, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6634615384615385}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6461074561403509}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.728525641025641}]
done 0 tensor(2.7922, grad_fn=<AddBackward0>)
curr best 100000000000
_++++++++++New Best++++____
tensor(2.7922, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6634615384615385}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6461074561403509}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.728525641025641}]
{'hidden_layers': [100, 100], 'attention_heads': [1, 1], 'embed_size': 0, 'dropout': 0.5, 'input_dropout': 0.5, 'shufflecol_chance': 0.1}
___________
++++++++



/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.8300, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6725961538461539}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6639254385964912}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7371794871794872}]
done 1 tensor(2.8300, grad_fn=<AddBackward0>)
curr best tensor(2.7922, grad_fn=<AddBackward0>)


/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

...

/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.7552, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.68125}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.7044956140350876}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7625}]
done 82 tensor(2.7552, grad_fn=<AddBackward0>)
curr best tensor(2.6872, grad_fn=<AddBackward0>)


/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.8200, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6600961538461538}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.6880482456140351}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.7612179487179487}]
done 83 tensor(2.8200, grad_fn=<AddBackward0>)
curr best tensor(2.6872, grad_fn=<AddBackward0>)
_________
+++++++++++
best stuff tensor(2.6872, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6817307692307693}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.7075109649122806}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.7807692307692308}]
{'hidden_layers': [600], 'attention_heads': [3], 'embed_size': 210, 'dropout': 0.9, 'input_dropout': 0.5, 'shufflecol_chance': 0.5}


DecisionAttentionModel(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=210, out_features=600, bias=True)
  )
  (batchnorm): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (final_layer): Linear(in_features=600, out_features=6, bias=True)
  (resize_layer): Linear(in_features=97, out_features=210, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=210, out_features=210, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((210,), eps=1e-05, elementwise_affine=True)
  )
  (activation): ReLU()
)

In [160]:
torch.save(decision_model,'../data/models/final_decision_model_' + decision_model.identifier + '.pt')
print('../data/models/final_decision_model_' + decision_model.identifier + '.pt')

../data/models/final_decision_model_statedecisions_input132_dims600_dropout0.5,0.9.pt


In [161]:
torch.save(model1,'../data/models/final_transition1_model_' + model1.identifier + '.pt')
torch.save(model2,'../data/models/final_transition2_model_' + model2.identifier + '.pt')
torch.save(model3,'../data/models/final_outcome_model_' + model3.identifier + '.pt')
print('../data/models/final_transition1_model_' + model1.identifier + '.pt')
print('../data/models/final_transition2_model_' + model2.identifier + '.pt')
print('../data/models/final_outcome_model_' + model3.identifier + '.pt')

../data/models/final_transition1_model_state1_input63_dims1000_dropout0.5,0.9.pt
../data/models/final_transition2_model_state2_input85_dims500,500_dropout0.5,0.9.pt
../data/models/final_outcome_model_state1_input83_dims500,500_dropout0.5,0.9.pt


In [ ]:
xatt = []
for att,xxdf in zip(list(attributions),xdf):
    new = pd.DataFrame(att.cpu().detach().numpy(),columns=xxdf.columns,index=xxdf.index)
    xatt.append(new)
attributions = pd.concat(xatt,axis=1)
attributions

In [ ]:
attributions.sum().sort_values()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.subplots(1,1,figsize=(100,100))
sns.heatmap(data=attributions.T,ax=fig[1])

In [ ]:
def breakup_state_models(state_model):
    #for state 1 and 2
    models = {}
    models['pd'] = lambda x: state_model(x)[0]
    models['nd'] = lambda x: state_model(x)[1]
    models['chemo'] = lambda x: state_model(x)[2]
    for i,dlt in enumerate(Const.dlt1):
        models[dlt] = lambda x: state_model(x)[3][:,i]
    return models

def breakup_outcome_models(omodel):
    models = {}
    for i,name in enumerate(Const.outcomes):
        models[name] = lambda x: omodel(x)[:,i].reshape(-1,1)
    return models

def get_all_models(m1,m2,m3):
    state1_models = breakup_state_models(m1)
    state2_models = breakup_state_models(m2)
    state3_models = breakup_outcome_models(m3)
    all_models = {}
    for i,sm in enumerate([state1_models,state2_models,state3_models]):
        for ii,m in sm.items():
            all_models[ii +  '_state' + str(i+1)] = m
    return all_models

all_models = get_all_models(model,model2,model3)
all_models

In [ ]:

def get_ytrue(name,df):
    outcomes=None
    value = None
    if name == 'pd_state1':
        outcomes = df[Const.primary_disease_states]
    elif name == 'pd_state2':
        outcomes = df[Const.primary_disease_states2]
    elif name == 'nd_state1':
        outcomes = df[Const.nodal_disease_states]
    elif name == 'nd_state2':
        outcomes = df[Const.nodal_disease_states2]
    elif name == 'chemo_state1':
        outcomes = df[Const.modifications]
    elif name == 'chemo_state2':
        outcomes = df[Const.ccs]   
    if outcomes is not None:
        value = outcomes.idxmax(axis=1)
    if 'DLT' in name:
        newname = name.replace('_state', ' ').replace('1','').strip()
        value = df[newname]
    if name.replace('_state3','') in Const.outcomes:
        value = df[name.replace('_state3','')]
    if value is None:
        print(name,df.columns)
    return value

def check_impact_of_decisions(model_dict,data):
    results = []
    #todo: this is wrong fix it
    ids = []
    df = data.get_data()
    outcomedict = {step: pd.concat(data.get_intermediate_outcomes(step=step),axis=1) for step in [1,2,3]}
    for decision in Const.decisions:
        for name, model in model_dict.items():
            step = int(name[-1])
            subset0 = dataset.get_input_state(step=step,fixed={decision: 0})
            subset1 = dataset.get_input_state(step=step,fixed={decision: 1})
            outcomes = outcomedict[step]
            ids = subset0.index.values
            x0 = df_to_torch(subset0)
            x1 = df_to_torch(subset1)
            y0 = model(x0).detach().cpu().numpy()
            y1 = model(x1).detach().cpu().numpy()
            original = data.get_input_state(step=step)
            xx = df_to_torch(original)
            yy = model(xx).detach().cpu().numpy()
            ytrue = get_ytrue(name,outcomes)
            if "DLT" in name:
                y0 = y0.argmax(axis=1).reshape(-1,1)
                y1 = y1.argmax(axis=1).reshape(-1,1)
                yy = yy.argmax(axis=1).reshape(-1,1)
                change = y0 - y1
                decision_change = (y0 != y1).astype(int)
            elif y0.shape[1] == 1:
                change = y1 - y0
                decision_change = np.abs((y0 > .5).astype(int) - (y1 > .5).astype(int))
            else:
                index = np.unravel_index(np.argmax(yy, axis=1), yy.shape)
                change = (y0[index] - y1[index]).reshape(-1,1)
                decision_change =  (y0.argmax(axis=1).reshape(-1,1) != y1.argmax(axis=1).reshape(-1,1)).astype(int)
                yy = yy.argmax(axis=1).reshape(-1,1)
                y1 = y1.argmax(axis=1).reshape(-1,1)
                y0 = y0.argmax(axis=1).reshape(-1,1)
            outcome = name.replace('_state','')
            for ii,pid in enumerate(ids):
                oo = ytrue.loc[pid]
                onew = y0[ii][0]
                original_decision = df.loc[pid,decision]
                if original_decision > 0:
                    onew = y0[ii][0]
                oname = Const.name_dict.get(name)
                if oname is not None:
                    onew = oname[onew]
                entry = {'id': pid, 'decision': decision,'outcome': outcome,'original_choice': original_decision, 'original_result': oo, 'alt_result': onew, 'change': change[ii][0], 'decision_change': decision_change[ii][0]}
                results.append(entry)
    return pd.DataFrame(results)

test = check_impact_of_decisions(all_models,dataset)
test

In [ ]:
data.get_data()['SD Primary 2'].sum()

In [ ]:
(test[test.outcome == 'pd2'].original_result == 'SD Primary 2').sum()

In [ ]:
check_impact_of_decisions(all_models,dataset).to_csv('../data/decision_impacts.csv')